In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("../data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.token_indexers import PretrainedBertIndexer

token_indexer = PretrainedBertIndexer(
    pretrained_model="bert-base-uncased",
    max_pieces=config.max_seq_len,
    do_lowercase=True,
 )
# apparently we need to truncate the sequence here, which is a stupid design decision
def tokenizer(s: str):
    return token_indexer.wordpiece_tokenizer(s)[:config.max_seq_len - 2]

02/07/2019 17:37:35 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/keitakurita/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:00, 295.56it/s]
251it [00:01, 168.06it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [[UNK],
  [UNK],
  the,
  edit,
  ##s,
  made,
  under,
  my,
  user,
  ##name,
  [UNK],
  [UNK],
  [UNK],
  were,
  reverted,
  ##?,
  [UNK],
  weren,
  ##',
  ##t,
  van,
  ##dal,
  ##isms,
  ##,,
  just,
  closure,
  on,
  some,
  [UNK],
  after,
  [UNK],
  voted,
  at,
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  [UNK],
  please,
  don,
  ##',
  ##t,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  [UNK],
  retired,
  now,
  ##.,
  ##8,
  ##9,
  ##.,
  ##20,
  ##5,
  ##.,
  ##38,
  ##.,
  ##27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.wordpiece_indexer.PretrainedBertIndexer at 0x1a2cbac128>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

We don't need to build the vocab: all that is handled by the token indexer

In [17]:
vocab = Vocabulary()

### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[  101,   100,   100,  ...,   102,     0,     0],
          [  101,   100,  1997,  ...,  1997,   100,   102],
          [  101,  1000,   100,  ...,  2062,  2411,   102],
          ...,
          [  101,   100,   100,  ..., 29625,   100,   102],
          [  101,   100,  1997,  ..., 12064,  2135,   102],
          [  101,   100,  5993,  ...,  2089,  2421,   102]]),
  'tokens-offsets': tensor([[ 1,  2,  3,  ..., 96,  0,  0],
          [ 1,  2,  3,  ..., 96, 97, 98],
          [ 1,  2,  3,  ..., 96, 97, 98],
          ...,
          [ 1,  2,  3,  ..., 96, 97, 98],
          [ 1,  2,  3,  ..., 96, 97, 98],
          [ 1,  2,  3,  ..., 96, 97, 98]]),
  'mask': tensor([[1, 1, 1,  ..., 1, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]])},
 'id': ['003a19c04c079bf7',
  '0098257c6952c9e3',
  '0048de0c9422f64f',
  

In [23]:
batch["tokens"]["tokens"]

tensor([[  101,   100,   100,  ...,   102,     0,     0],
        [  101,   100,  1997,  ...,  1997,   100,   102],
        [  101,  1000,   100,  ...,  2062,  2411,   102],
        ...,
        [  101,   100,   100,  ..., 29625,   100,   102],
        [  101,   100,  1997,  ..., 12064,  2135,   102],
        [  101,   100,  5993,  ...,  2089,  2421,   102]])

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([11, 100])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True, # conserve memory
)
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                            # we'll be ignoring masks so we'll need to set this to True
                                                           allow_unmatched_keys = True)

02/07/2019 17:37:42 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/keitakurita/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
02/07/2019 17:37:42 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/keitakurita/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/hy/1czs1y5j2d58zgkqx6w_wnpw0000gn/T/tmpm99yyacp
02/07/2019 17:37:48 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads"

In [28]:
BERT_DIM = word_embeddings.get_output_dim()

class BertSentencePooler(Seq2VecEncoder):
    def forward(self, embs: torch.tensor, 
                mask: torch.tensor=None) -> torch.tensor:
        # extract first token tensor
        return embs[:, 0]
    
    @overrides
    def get_output_dim(self) -> int:
        return BERT_DIM
    
encoder = BertSentencePooler(vocab)

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[  101,   100,   100,  ...,   102,     0,     0],
         [  101,   100,  1997,  ...,  1997,   100,   102],
         [  101,  1000,   100,  ...,  2062,  2411,   102],
         ...,
         [  101,   100,   100,  ..., 29625,   100,   102],
         [  101,   100,  1997,  ..., 12064,  2135,   102],
         [  101,   100,  5993,  ...,  2089,  2421,   102]]),
 'tokens-offsets': tensor([[ 1,  2,  3,  ..., 96,  0,  0],
         [ 1,  2,  3,  ..., 96, 97, 98],
         [ 1,  2,  3,  ..., 96, 97, 98],
         ...,
         [ 1,  2,  3,  ..., 96, 97, 98],
         [ 1,  2,  3,  ..., 96, 97, 98],
         [ 1,  2,  3,  ..., 96, 97, 98]]),
 'mask': tensor([[1, 1, 1,  ..., 1, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])}

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[ 0.4729,  0.2816,  0.4229, -0.0508,  0.2860,  0.1277],
        [ 0.4567,  0.1642,  0.1731, -0.0892,  0.0180, -0.0179],
        [ 0.4481,  0.1152,  0.1730, -0.2053, -0.1605, -0.1381],
        [ 0.3364,  0.1633,  0.1099, -0.1427, -0.0838,  0.0581],
        [ 0.3577,  0.1493,  0.0643, -0.1915, -0.0968, -0.1219],
        [ 0.2557,  0.1389,  0.2810, -0.1334, -0.0666, -0.0243],
        [ 0.2137,  0.1822,  0.2200, -0.1423, -0.0362, -0.1285],
        [ 0.5856,  0.1423,  0.3972, -0.0657,  0.0962,  0.1120],
        [ 0.3897,  0.2418,  0.1570, -0.2679,  0.0226, -0.1083],
        [ 0.1805, -0.0639, -0.0843, -0.0652, -0.0810, -0.0309],
        [ 0.2778,  0.1103,  0.3753,  0.0957,  0.4034, -0.0487]],
       grad_fn=<AddmmBackward>)

In [36]:
model(**batch)

{'class_logits': tensor([[ 0.4487,  0.2121,  0.3770, -0.0024, -0.0786, -0.1584],
         [ 0.3425,  0.1704,  0.2082, -0.1272, -0.0710, -0.2600],
         [ 0.3889,  0.2092,  0.3206, -0.0456, -0.0397, -0.1992],
         [ 0.3932, -0.0668,  0.1999, -0.0447,  0.2420,  0.0691],
         [ 0.3591,  0.2240,  0.1109, -0.1412, -0.1365, -0.1329],
         [ 0.2854,  0.1720,  0.0455, -0.0622, -0.1601,  0.0505],
         [ 0.1628,  0.0641,  0.1516, -0.0599, -0.0546, -0.0176],
         [ 0.4083,  0.0808,  0.1382, -0.1770, -0.0371,  0.0616],
         [ 0.3897,  0.2182, -0.0965, -0.2455, -0.0945, -0.0205],
         [ 0.1571,  0.0584,  0.1086, -0.1595, -0.0177, -0.1803],
         [ 0.1892,  0.1309,  0.4160, -0.0852,  0.1782, -0.2983]],
        grad_fn=<AddmmBackward>),
 'loss': tensor(0.7260, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)}

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.7254, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[]

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

02/07/2019 17:38:09 - INFO - allennlp.training.trainer -   Beginning training.
02/07/2019 17:38:09 - INFO - allennlp.training.trainer -   Epoch 0/1
02/07/2019 17:38:09 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 1287.417856
02/07/2019 17:38:09 - INFO - allennlp.training.trainer -   Training
loss: 0.7036 ||: 100%|██████████| 5/5 [01:28<00:00, 18.71s/it]
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -                     Training |  Validation
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -   loss          |     0.704  |       N/A
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -   cpu_memory_MB |  1287.418  |       N/A
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -   Epoch duration: 00:01:29
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -   Estimated training time remaining: 0:01:29
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -   Epoch 1/1
02/07/2019 17:39:38 - INFO - allennlp.training.trainer -   Peak CPU memor

# Generating Predictions

In [44]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [45]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [46]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 

100%|██████████| 4/4 [00:44<00:00, 11.04s/it]
